In [1]:
cd /home

/home


In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import mlflow
import os

from sklearn.metrics import r2_score, mean_squared_error
from src.utils import get_metrics

from presentation.pipelines.steps.model_design import load_pt_model
from presentation.pipelines.steps.load_data import build_loader 


os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

%load_ext autoreload
%autoreload 2

2024-04-02 21:34:33.832899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
root = './presentation/results/lfgr/2024-03-25_18-27-29/'

astromer, config = load_pt_model(os.path.join(root, 'pretraining'))

[INFO] Masking Query tokens only
[INFO] Masking Query tokens only


In [13]:
config

{'exp_name': 'lfgr',
 'data': './data/records/macho_clean/',
 'checkpoint': '-1',
 'gpu': '1',
 'debug': False,
 'scheduler': False,
 'arch': 'base',
 'num_layers': 2,
 'num_heads': 4,
 'head_dim': 64,
 'pe_dim': 256,
 'pe_base': 1000,
 'pe_exp': 2,
 'mixer': 128,
 'dropout': 0.0,
 'm_alpha': -1000000000.0,
 'mask_format': 'Q',
 'use_leak': True,
 'repeat': 1,
 'lr': 1e-05,
 'bs': 1024,
 'patience': 20,
 'num_epochs': 10000,
 'window_size': 200,
 'probed': 0.5,
 'rs': 0.2,
 'same': 0.2,
 'nsp_prob': 0.5,
 'correct_loss': True,
 'loss_format': 'rmse'}

In [14]:
data_path = './data/records/alcock/fold_0/alcock_20/'
loaders = build_loader(data_path, 
                       config, 
                       batch_size=32,
                       clf_mode=True,
                       sampling=False,
                       normalize='zero-mean')

Classification Mode
[INFO] Probed: 1.00 Random: 0.00 Same: 0.00
[INFO] Repeating dataset x1 times
[INFO] Loading PT task: Masking
[INFO] Probed: 1.00 Random: 0.00 Same: 0.00
[INFO] Repeating dataset x1 times
[INFO] Loading PT task: Masking
[INFO] Probed: 1.00 Random: 0.00 Same: 0.00
[INFO] Repeating dataset x1 times
[INFO] Loading PT task: Masking


In [15]:
for x, y in loaders['train'].take(1):
    print(x['mask_in'])
    print(y)

tf.Tensor(
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]], shape=(32, 200, 1), dtype=float32)
tf.Tensor(
[[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0